In [1]:
--- Libraries:
arguments = require 'Settings.arguments'
constants = require 'Settings.constants'
card_to_string = require 'Game.card_to_string_conversion'
card_tools = require 'Game.card_tools'
game_settings = require 'Settings.game_settings'
Plot = require 'itorch.Plot'
nninit = require 'nninit'
require 'torch'
require 'math'
require 'Tree.tree_builder'
require 'Tree.tree_data_generation'
require 'Tree.tree_values'
require 'Tree.tree_visualiser'
require 'nn'
require 'Tree.tree_cfr'
require 'Tree.tree_cfr2'
require 'nngraph'
require 'optim'
require 'image'
require 'NN.strategy_net_builder'
require 'NN.strategy_net_trainer'
require 'Tree.tree_warm_start'
---require 'Adversarials.exploitability'
require 'Adversarials.exploitabilityVS'



In [2]:
function get_first_mask_tensor(index)
    local output = torch.range(1,6)
    if index ==1 then
        return torch.range(2,6)
    end
    local out1 = output:clone():narrow(1,1,index-1)
    local out2 = output:clone():narrow(1,index+1,6-index)
    local result = out1:cat(out2)
    return result
end

function get_second_mask_tensor(input_tensor,index)
    local result = {}
    local input = input_tensor:clone()
    for i = 1,input:size(1) do
        if input[i] ~= index then
            table.insert(result,input[i])
        end
    end
    return torch.Tensor(result)
end


function construct_possible_game_states()
    local states_tensor = torch.Tensor()
    local first_card_tensor = torch.Tensor({1,3,5})
    for i = 1,first_card_tensor:size(1) do
        local second_card_tensor = get_first_mask_tensor(first_card_tensor[i])
        for j =1, second_card_tensor:size(1) do
            local third_card_tensor = get_second_mask_tensor(second_card_tensor,second_card_tensor[j])
            for k = 1,third_card_tensor:size(1) do
                local states = torch.Tensor({first_card_tensor[i],second_card_tensor[j],third_card_tensor[k]}):view(1,3)
                states_tensor = states_tensor:cat(states,1)
            end
        end
    end
    return states_tensor
end

function distribute_cards(cards_tensor)
    local cards = {}
    cards.P1 = cards_tensor[1]
    cards.P2 = cards_tensor[2]
    cards.board = cards_tensor[3]
    return cards
end

In [3]:
--- Create the tree
builder = PokerTreeBuilder()
--- Parameters for the tree
params = {}
params.root_node = {}
params.root_node.board = card_to_string:string_to_board('')
params.root_node.street = 1
params.root_node.current_player = constants.players.P1
params.root_node.bets = arguments.Tensor{200,200}
--- CFR Solver
starting_ranges = arguments.Tensor(constants.players_count, game_settings.card_count)
starting_ranges[1]:copy(card_tools:get_uniform_range(params.root_node.board))
starting_ranges[2]:copy(card_tools:get_uniform_range(params.root_node.board))
--- exploitability
exploit = TreeValues()

In [4]:
--- Initialize trees
tree1 = builder:build_tree(params)
tree2 = builder:build_tree(params)

In [5]:
---visualiser = TreeVisualiser()
---visualiser:graphviz(tree1, "zero_blind")

In [6]:
solver2 = TreeCFR()
solver2.cfr_skip = 0
solver2:run_cfr(tree2,starting_ranges,500)

In [7]:
--- NNWarm Starting
dataNN = TreeData(tree2)
dataNN:get_training_set(tree2,1)

In [28]:
tree_eval = StrategyEvaluator(tree1,tree2)

In [99]:
#range_games

 3
[torch.LongStorage of size 1]



In [ ]:
min_pot = 200
number_of_times = 500
range_games = torch.range(0,number_of_times,20)
pots_won_tensor = torch.FloatTensor(1,range_games:size(1))
avg_gain_per_game = torch.FloatTensor(1,range_games:size(1))
range_games[1] = 1
for i = 1,range_games:size(1) do
    tree_eval:play_all_combinations_n_times(tree1,tree2,range_games[i])
    pots_won_tensor[{1,i}] = tree_eval.A2_pot_matrix:sum()/(min_pot*number_of_times)
    avg_gain_per_game[{1,i}] = tree_eval.A2_pot_matrix:sum()/(number_of_times*60)
    print("Total number of pots won : ",tree_eval.A2_pot_matrix:sum()/(min_pot*number_of_times))
    print("average amount won per game",tree_eval.A2_pot_matrix:sum()/(number_of_times*60))
end

Total number of pots won : 	0.102
average amount won per game	0.34


Total number of pots won : 	1.51
average amount won per game	5.0333333333333


Total number of pots won : 	2.866
average amount won per game	9.5533333333333


Total number of pots won : 	4.122
average amount won per game	13.74


Total number of pots won : 	4.982
average amount won per game	16.606666666667


...

Total number of pots won : 	9.538
average amount won per game	31.793333333333


Total number of pots won : 	11.688
average amount won per game	38.96


Total number of pots won : 	12.556
average amount won per game	41.853333333333


Total number of pots won : 	14.074
average amount won per game	46.913333333333


Total number of pots won : 	15.42
average amount won per game	51.4


In [131]:
avg_won = avg_gain_per_game:clone():cdiv(range_games)

In [135]:
x = torch.range(1,avg_gain_per_game:size(2))
local plot = Plot()

---plot:line(x,avg_gain_per_game:view(-1),'black' ,'Original CFR in the paper')
plot:line(x,avg_won:view(-1),'blue' ,'Original CFR in the paper'):draw()
---plot:line(x,pots_won_tensor:view(-1),'red' ,'No Warm start CFR'):draw()


In [75]:
tree_eval:display_results()

ties: 	615
P1 won: 	4882
P2 won: 	6503



In [76]:
print(tree_eval.A2_pot_matrix:sum())
print(tree_eval.A2_pot_matrix:std())

In [77]:
print("Total number of pots won : ",tree_eval.A2_pot_matrix:sum()/min_pot)

1375400
544.20372980212


Total number of pots won : 	6877


In [78]:
print("average amount won per game",tree_eval.A2_pot_matrix:sum()/(number_of_times*60))

average amount won per game	114.61666666667


In [21]:
avg_per_hand = tree_eval.A2_pot_matrix:mean(2)

In [22]:
plot = Plot():circle(torch.range(1,avg_per_hand:size(1)),avg_per_hand:view(-1))
plot:legend(true):title('Player 2 pot distribution, 10000 games, each possible card dealt'):draw()


In [ ]:
cards = {}
cards.P1 = 1
cards.P2 = 2
cards.board = 4
number_of_games = 100

In [ ]:
tree_eval.A2_pot_tensor

In [ ]:
print(tree_eval.A2_pot_tensor:mean())
print(tree_eval.A2_pot_tensor:std())

In [ ]:
tree_eval:display_results()

In [ ]:
local pot_table = {}
local size_sample = 50
for i = 1,50 do
    tree_eval:play_several_games(tree1,tree2,size_sample)
    table.insert(pot_table,tree_eval.A2_pot/size_sample)
end
print("player_2 won: ",torch.FloatTensor(pot_table):mean())

In [ ]:
pot_table_A2 = {}
local number_of_games = 1000
local cards_tensor = construct_possible_game_states()
for i = 1,cards_tensor:size(1) do
    local cards = distribute_cards(cards_tensor[i])
    tree_eval:play_deterministic_games(tree1,tree2,number_of_games,cards)
    table.insert(pot_table_A2,tree_eval.A2_pot)
end
print("player_2 won: ",torch.FloatTensor(pot_table_A2):mean())
print("player_2 std: ",torch.FloatTensor(pot_table_A2):std())
tree_eval:display_results()

In [ ]:
torch.sqrt(60)*(1803/150716)

In [ ]:
plot = Plot():histogram(torch.FloatTensor(pot_table_A2)):draw()

In [ ]:
pot_table1 = {}
local number_of_games = 10
local cards_tensor = construct_possible_game_states()
for i = 1,cards_tensor:size(1) do
    for j = 1,number_of_games do
        local cards = distribute_cards(cards_tensor[i])
        tree_eval:play_deterministic_games(tree1,tree2,number_of_plays,cards)
        table.insert(pot_table1,tree_eval.A2_pot)
    end
end
print("player_2 won: ",torch.FloatTensor(pot_table1):mean())
print("player_2 std: ",torch.FloatTensor(pot_table1):std())

In [ ]:
pot_table1 = {}
local number_of_games = 20
local cards_tensor = construct_possible_game_states()
for i = 1,cards_tensor:size(1) do
    for j = 1,number_of_games do
        local cards = distribute_cards(cards_tensor[i])
        tree_eval:play_deterministic_games(tree1,tree2,number_of_plays,cards)
        table.insert(pot_table1,tree_eval.A2_pot)
    end
end
print("player_2 won: ",torch.FloatTensor(pot_table1):mean())
print("player_2 std: ",torch.FloatTensor(pot_table1):std())

In [ ]:
pot_table1 = {}
local number_of_games = 50
local cards_tensor = construct_possible_game_states()
for i = 1,cards_tensor:size(1) do
    for j = 1,number_of_games do
        local cards = distribute_cards(cards_tensor[i])
        tree_eval:play_deterministic_games(tree1,tree2,number_of_plays,cards)   --- --> CHECK NUMBER OF GAMES!
        table.insert(pot_table1,tree_eval.A2_pot)
    end
end
print("player_2 won: ",torch.FloatTensor(pot_table1):mean())
print("player_2 std: ",torch.FloatTensor(pot_table1):std())

In [ ]:
 cardsInv = {}
cardsInv.P1 = cards.P2
cardsInv.P2 = cards.P1
cardsInv.board = cards.board

In [ ]:
pot_table2 = {}
local size_sample = 100
for i = 1,20 do
    tree_eval:play_deterministic_games(tree1,tree2,size_sample,cardsInv)
    table.insert(pot_table2,tree_eval.A2_pot/size_sample)
end
print("player_2 won: ",torch.FloatTensor(pot_table2):mean())
print("player_2 std: ",torch.FloatTensor(pot_table2):std())

In [ ]:
---plot = Plot():histogram(torch.FloatTensor(pot_table1)):draw()

In [ ]:
---plot = Plot():histogram(torch.FloatTensor(pot_table2)):draw()

In [ ]:
x[1]